# 定義

## 環境定義

In [ ]:
$vmName       = 'Test-JupyterPowerShell'
$isoPath      = 'C:\pathto\iso\WindowsClients\Win11_24H2_Japanese_x64.iso'
$vhdPath      = "$VMName.vhdx"
$vhdSize      = 64GB
$vmGeneration = 2
$vmMemory     = 4GB
$vmProcessor  = 2
$vmSecureBoot = $true
$vmCheckpoint = $false
$vmSwitchName   = 'Default Switch'
$autounattend = '.\autounattend'
$userName     = 'sakaztk'
$password     = 'Passw0rd!'
$locale       = 'ja-JP'
$keyboard     = '0411:00000411'
$productKey   = 'VK7JG-NPHTM-C97JM-9MPGT-3V66T'

## 自動応答ファイル

### フォルダ作成

In [ ]:
if (-not(Test-Path $autounattend)){New-Item -ItemType Directory -Name $autounattend}

### 初回ログオンコマンド部分

In [ ]:
$logonCommands = @(
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Start-Sleep -Seconds 10; Get-NetConnectionProfile | Set-NetConnectionProfile -NetworkCategory Private}"'
        Description = 'Set network to Private'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Enable-PSRemoting -Force}"'
        Description = 'Enable PSRemoting'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Set-ItemProperty -Path "HKLM:\SYSTEM\CurrentControlSet\Control\Terminal Server" -Name fDenyTSConnections -Value 0}"'
        Description = 'Enable Remote Desktop'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Enable-NetFirewallRule -DisplayGroup "Remote Desktop"}"'
        Description = 'Enable Remote Desktop Rules'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Disable-LocalUser -Name "Administrator"}"'
        Description = 'Disable Built-in Administrator'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Set-ItemProperty "HKLM:\SYSTEM\CurrentControlSet\Services\i8042prt\Parameters" -Name "LayerDriver JPN" -Value "kbd106.dll"}"'
        Description = 'Set keyboard to 106'
    }
    @{
        Command = 'powershell.exe -ExecutionPolicy Bypass -Command "&amp; {Restart-Computer -Force}"'
        Description = 'Reboot'
    }
)
$xmlLogonCommands = @()
$order = 1
foreach ($item in $logonCommands) {
    $xmlLogonCommands += "    <SynchronousCommand wcm:action=`"add`">"
    $xmlLogonCommands += "      <Order>$order</Order>"
    $xmlLogonCommands += "      <CommandLine>$($item.Command)</CommandLine>"
    $xmlLogonCommands += "      <Description>$($item.Description)</Description>"
    $xmlLogonCommands += "    </SynchronousCommand>"
    $order++
}
$xmlLogonCommands

### xml生成

In [ ]:
$unattendXml = @"
<?xml version="1.0" encoding="utf-8"?>
<unattend xmlns="urn:schemas-microsoft-com:unattend">
  <settings pass="windowsPE">
    <component name="Microsoft-Windows-International-Core-WinPE" processorArchitecture="amd64" publicKeyToken="31bf3856ad364e35" language="neutral" versionScope="nonSxS" xmlns:wcm="http://schemas.microsoft.com/WMIConfig/2002/State">
      <InputLocale>$locale</InputLocale>
      <SystemLocale>$locale</SystemLocale>
      <UILanguage>$locale</UILanguage>
      <UILanguageFallback>en-US</UILanguageFallback>
      <UserLocale>$locale</UserLocale>
    </component>
    <component name="Microsoft-Windows-Setup" processorArchitecture="amd64" publicKeyToken="31bf3856ad364e35" language="neutral" versionScope="nonSxS" xmlns:wcm="http://schemas.microsoft.com/WMIConfig/2002/State">
      <DiskConfiguration>
        <Disk wcm:action="add">
          <DiskID>0</DiskID>
          <WillWipeDisk>true</WillWipeDisk>
          <CreatePartitions>
            <CreatePartition wcm:action="add">
              <Order>1</Order>
              <Type>EFI</Type>
              <Size>100</Size>
            </CreatePartition>
            <CreatePartition wcm:action="add">
              <Order>2</Order>
              <Type>MSR</Type>
              <Size>16</Size>
            </CreatePartition>
            <CreatePartition wcm:action="add">
              <Order>3</Order>
              <Type>Primary</Type>
              <Extend>true</Extend>
            </CreatePartition>
          </CreatePartitions>
          <ModifyPartitions>
            <ModifyPartition wcm:action="add">
              <Order>1</Order>
              <PartitionID>1</PartitionID>
              <Format>FAT32</Format>
              <Label>System</Label>
            </ModifyPartition>
            <ModifyPartition wcm:action="add">
              <Order>2</Order>
              <PartitionID>2</PartitionID>
            </ModifyPartition>
            <ModifyPartition wcm:action="add">
              <Order>3</Order>
              <PartitionID>3</PartitionID>
              <Format>NTFS</Format>
              <Label>Windows</Label>
              <Letter>C</Letter>
            </ModifyPartition>
          </ModifyPartitions>
        </Disk>
        <WillShowUI>OnError</WillShowUI>
      </DiskConfiguration>
      <ImageInstall>
        <OSImage>
          <InstallFrom>
            <MetaData wcm:action="add">
              <Key>/IMAGE/INDEX</Key>
              <Value>1</Value>
            </MetaData>
          </InstallFrom>
          <InstallTo>
            <DiskID>0</DiskID>
            <PartitionID>3</PartitionID>
          </InstallTo>
          <WillShowUI>OnError</WillShowUI>
        </OSImage>
      </ImageInstall>
      <UserData>
        <AcceptEula>true</AcceptEula>
        <FullName>$userName</FullName>
        <Organization>MyOrg</Organization>
        <ProductKey>
          <Key>$productKey</Key>
        </ProductKey>
      </UserData>
    </component>
  </settings>
  <settings pass="oobeSystem">
    <component name="Microsoft-Windows-International-Core" processorArchitecture="amd64" publicKeyToken="31bf3856ad364e35" language="neutral" versionScope="nonSxS" xmlns:wcm="http://schemas.microsoft.com/WMIConfig/2002/State">
      <InputLocale>$locale</InputLocale>
      <SystemLocale>$locale</SystemLocale>
      <UILanguage>$locale</UILanguage>
      <UserLocale>$locale</UserLocale>
    </component>
    <component name="Microsoft-Windows-Shell-Setup" processorArchitecture="amd64" publicKeyToken="31bf3856ad364e35" language="neutral" versionScope="nonSxS" xmlns:wcm="http://schemas.microsoft.com/WMIConfig/2002/State">
      <OOBE>
        <HideEULAPage>true</HideEULAPage>
        <HideOEMRegistrationScreen>true</HideOEMRegistrationScreen>
        <HideWirelessSetupInOOBE>true</HideWirelessSetupInOOBE>
        <HideOnlineAccountScreens>true</HideOnlineAccountScreens>
        <SkipMachineOOBE>true</SkipMachineOOBE>
        <SkipUserOOBE>true</SkipUserOOBE>
        <NetworkLocation>Work</NetworkLocation>
        <ProtectYourPC>1</ProtectYourPC>
      </OOBE>
      <UserAccounts>
        <LocalAccounts>
          <LocalAccount wcm:action="add">
            <Name>$userName</Name>
            <Group>Administrators</Group>
            <Password>
              <Value>$password</Value>
              <PlainText>true</PlainText>
            </Password>
          </LocalAccount>
        </LocalAccounts>
      </UserAccounts>
      <AutoLogon>
        <Username>$userName</Username>
        <Enabled>true</Enabled>
        <Password>
          <Value>$password</Value>
          <PlainText>true</PlainText>
        </Password>
      </AutoLogon>
      <RegisteredOrganization>MyOrg</RegisteredOrganization>
      <RegisteredOwner>$userName</RegisteredOwner>
      <TimeZone>Tokyo Standard Time</TimeZone>
      <FirstLogonCommands>
        $xmlLogonCommands
      </FirstLogonCommands>
    </component>
  </settings>
  <settings pass="specialize">
    <component name="Microsoft-Windows-International-Core" processorArchitecture="amd64" publicKeyToken="31bf3856ad364e35" language="neutral" versionScope="nonSxS" xmlns:wcm="http://schemas.microsoft.com/WMIConfig/2002/State">
      <InputLocale>$locale</InputLocale>
      <SystemLocale>$locale</SystemLocale>
      <UILanguage>$locale</UILanguage>
      <UserLocale>$locale</UserLocale>
    </component>
  </settings>
  <cpi:offlineImage cpi:source="wim://sources/install.wim#Windows 11 Pro" xmlns:cpi="urn:schemas-microsoft-com:cpi" />
</unattend>
"@ | Out-File -FilePath '.\autounattend\autounattend.xml' -Encoding utf8 -Force -Verbose

# 構築

## VHD作成

In [ ]:
New-VHD -Path $vhdPath -SizeBytes $vhdSize -Dynamic -Verbose

## VM作成

In [ ]:
New-VM -Name $vmName -MemoryStartupBytes $vmMemory -Generation $vmGeneration -SwitchName $vmSwitchName -Verbose

In [ ]:
Set-VM -VMName $vmName -AutomaticCheckpointsEnabled $vmCheckpoint -Verbose

In [ ]:
Set-VMProcessor -VMName $vmName -Count $vmProcessor -Verbose

In [ ]:
Set-VMFirmware -VMName $vmName -EnableSecureBoot On -Verbose

In [ ]:
Set-VMKeyProtector -VMName $vmName -NewLocalKeyProtector -Verbose

In [ ]:
Enable-VMTPM -VMName $vmName -Verbose

In [ ]:
Add-VMHardDiskDrive -VMName $vmName -Path $vhdPath -Verbose

In [ ]:
$dvdDrive = Add-VMDvdDrive -VMName $vmName -Path $isoPath -Verbose -Passthru

## 自動応答ファイルをisoファイルに格納してDVDとしてマウントする
WindowsADKのoscdimg.exeを使用

In [ ]:
$packageId = 'Microsoft.WindowsADK'
$installed = winget list --id $packageId | Select-String $packageId
$existsUpdate = winget update | Select-String $packageId
if(-not($installed)) {
    winget install $packageId --accept-package-agreements --accept-source-agreements
}
elseif($existsUpdate) {
    winget update $packageId --accept-package-agreements --accept-source-agreements
}
else{'Windows ADK is up to date.'}

In [ ]:
$tempISO = 'autounattend.iso'
if (Test-Path $tempISO) { Remove-Item $tempISO -Force -Verbose }

In [ ]:
$cd = "${env:ProgramFiles(x86)}\Windows Kits\10\Assessment and Deployment Kit\Deployment Tools\amd64\Oscdimg\oscdimg.exe"
& $cd -n -m $autounattend $tempISO

In [ ]:
$auDrive = Add-VMDvdDrive -VMName $vmName -Path $tempISO -Verbose -Passthru

## BootOrderを変更

In [ ]:
$hdDrive = Get-VMHardDiskDrive -VMName $vmName

In [ ]:
$networkAdapter = Get-VMNetworkAdapter -vmName $vmName

In [ ]:
Set-VMFirmware -VMName $vmName -BootOrder $auDrive, $dvdDrive, $hdDrive, $networkAdapter -Verbose

In [ ]:
(Get-VMFirmware -VMName $vmName).BootOrder.Device.Path

## VM起動
OSインストールする

In [ ]:
Start-VM -Name $VMName -Verbose -Passthru

In [ ]:
$retry = 0
while ($retry -lt 300) {
    try {
        $dsIpAddress = (Get-VMNetworkAdapter -VMName $vmName | Where-Object {$_.SwitchName -eq 'Default Switch'}).IPAddresses |
        Where-Object {($_ -match '^\d{1,3}(\.\d{1,3}){3}$') -and ($_ -notlike '169.254*') -and ($_ -ne '0.0.0.0')}
        Test-WSMan $dsIpAddress -ErrorAction Stop
        break
    } catch {
        Start-Sleep -Seconds 10
        $retry++
    }
}

## DVDドライブ調整

### インストーラーはアンマウント

In [ ]:
Get-VMDvdDrive -VMName $vmName | Where-Object {
    $_.Path -like "*$isoPath*"
} | Set-VMDvdDrive -Path $null -Verbose

### 自動応答ファイルはドライブ削除

In [ ]:
Get-VMDvdDrive -VMName $vmName | Where-Object {
    $_.Path -like "*$tempISO*"
} | Remove-VMDvdDrive -Verbose